# Library

In [4]:
# Import Library
import os

## Universal Data Processing
import numpy as np
import pandas as pd

## Regular Expression for Text Data
import re

## JSON Files Manipulation
import json
from pathlib import Path

In [48]:
# show all columns
pd.set_option("display.max_columns", None)   

# auto-detect width
pd.set_option("display.width", None)         

# don't truncate cell content
pd.set_option("display.max_colwidth", None)  
pd.set_option('display.max_rows', None)

# Load and Parse Dataset

## Load Data

In [5]:
# Path
path = Path("C:/03. Other/op-shops/Datasets/opshops_with_coords.json")

# Load dataset
all_data = []

print(f"Loading file: {path}")

with open(path, "r", encoding="utf-8") as json_file:
    data = json.load(json_file)
    all_data.append(data)

print("Files loaded:", len(all_data))
print("Records in file:", len(all_data[0]) if all_data else 0)

Loading file: C:\03. Other\op-shops\Datasets\opshops_with_coords.json
Files loaded: 1
Records in file: 2297


In [47]:
# Display first few records
all_data[0][:2]

[['Koo Wee Rup and District Lions Community Opportunity Shop',
  '290 Rossiter Road, Koo Wee Rup',
  -38.1993407,
  145.4912486,
  'Mon - Fri 10am-4pm',
  'Sat 9am-12pm',
  'Sun 10am-3pm selected Sundays'],
 ['Rose Lodge Opportunity Shop',
  '42 McBride Avenue, Wonthaggi, 3996',
  -38.6063217,
  145.5909252,
  'Mon - Fri 9am - 4pm',
  'Sat 9am - 12pm']]

## Convert to Dataframe

In [36]:
# Define custom headers
columns = [
    "name",
    "address",
    "latitude",
    "longitude",
    "hours_1",
    "hours_2",
    "hours_3"
    ]

# Normalize rows: pad with None so all rows have same length
max_len = len(columns)

normalized = []

for row in data:
    row = list(row)
    if len(row) < max_len:
        row = row + [None] * (max_len - len(row))
    else:
        row = row[:max_len]  # trim if too long
    normalized.append(row)

# Create DataFrame
df_opshop = pd.DataFrame(normalized, columns=columns)

# Show only first 5 rows
df_opshop.head(5)

,name,address,latitude,longitude,hours_1,hours_2,hours_3
0,Koo Wee Rup and District Lions Community Opportunity Shop,"290 Rossiter Road, Koo Wee Rup",-38.199341,145.491249,Mon - Fri 10am-4pm,Sat 9am-12pm,Sun 10am-3pm selected Sundays
1,Rose Lodge Opportunity Shop,"42 McBride Avenue, Wonthaggi, 3996",-38.606322,145.590925,Mon - Fri 9am - 4pm,Sat 9am - 12pm,NaN
2,The Green Shed Underground,"Basement level, 34 E Row Canberra ACT 2601 Australia",NaN,NaN,Tue - Fri 10:30am - 6pm,Sat 10:30am - 4pm,NaN
3,Hopeworks Seaford Op Shop,"2/36 Hartnett Dr 3198 Seaford, Victoria",-38.113295,145.141560,Mon - Fri 9am-4pm,Sat 9am-3pm,Sun CLOSED
4,The Collective,"270 Rossiter Road, Koo Wee Rup",-38.199690,145.490323,Mon - Tue 10am-4pm,Fri - Sun 10am-4pm,NaN


## Parse Data

### Address

In [42]:
# Display Address Column
df_opshop

,name,address,latitude,longitude,hours_1,hours_2,hours_3
0,Koo Wee Rup and District Lions Community Opportunity Shop,"290 Rossiter Road, Koo Wee Rup",-38.199341,145.491249,Mon - Fri 10am-4pm,Sat 9am-12pm,Sun 10am-3pm selected Sundays
1,Rose Lodge Opportunity Shop,"42 McBride Avenue, Wonthaggi, 3996",-38.606322,145.590925,Mon - Fri 9am - 4pm,Sat 9am - 12pm,NaN
2,The Green Shed Underground,"Basement level, 34 E Row Canberra ACT 2601 Australia",NaN,NaN,Tue - Fri 10:30am - 6pm,Sat 10:30am - 4pm,NaN
3,Hopeworks Seaford Op Shop,"2/36 Hartnett Dr 3198 Seaford, Victoria",-38.113295,145.141560,Mon - Fri 9am-4pm,Sat 9am-3pm,Sun CLOSED
4,The Collective,"270 Rossiter Road, Koo Wee Rup",-38.199690,145.490323,Mon - Tue 10am-4pm,Fri - Sun 10am-4pm,NaN
5,All Nations Christian Church Op Shop,"170 Seacombe Rd, Seaview Downs",-35.029295,138.541582,Wed - Thu 10-4pm,NaN,NaN
6,Anglican Op Shop,"Shop 1 100 - 106 Old Pacific Hwy, Oxenford 4210",NaN,NaN,Mon - Fri 8.30 - 4.30,Sat 8.30 -1,Sun Closed
7,Destiny Inc. Op Shop,6 jervois Street Albert park,-34.876736,138.526764,Tue - Fri 9:30-4:00,NaN,NaN
8,Beach End Op Shop Mornington,70 McLaren Place Mornington Victoria 3931 Australia,NaN,NaN,Mon - Fri 10am - 4pm,Sat 10am - 2pm,Sun CLOSED
9,Starting Point Op Shop,"6 Prossers Forest Road, Ravenswood, Launceston, TAS, 7250",-41.416195,147.179290,Mon - Thu 10am-2pm,Fri - Sun Closed,NaN


Briefly we can see there are several ***issues*** on the data:   
   
*  `name` are consists of address.   
   
*  `address` are consists of hours opening.   
   
*  `hours_#` is in initial format (e.g. `M-F 09.00 - 10.00`) rather than usual format in majority (e.g. `Mon - Fri 09.00 - 10.00`)   
    
***To-Do***:    
   
*  Split and flag the datasets into two category: `Full-data` and `Lat/Long = NaN`.   
   *  Assumption that the records in `Full-data` will be uniform so the parsing will be similar.
   *  The records in `Lat/Long = NaN` that have many anomalies and require more analysis of the format and format the appropriate parse approach.

In [52]:
# Full-data
df_fulldata = df_opshop[df_opshop["latitude"].notna() & df_opshop["longitude"].notna()]

df_fulldata.head()

,name,address,latitude,longitude,hours_1,hours_2,hours_3
0,Koo Wee Rup and District Lions Community Opportunity Shop,"290 Rossiter Road, Koo Wee Rup",-38.199341,145.491249,Mon - Fri 10am-4pm,Sat 9am-12pm,Sun 10am-3pm selected Sundays
1,Rose Lodge Opportunity Shop,"42 McBride Avenue, Wonthaggi, 3996",-38.606322,145.590925,Mon - Fri 9am - 4pm,Sat 9am - 12pm,NaN
3,Hopeworks Seaford Op Shop,"2/36 Hartnett Dr 3198 Seaford, Victoria",-38.113295,145.141560,Mon - Fri 9am-4pm,Sat 9am-3pm,Sun CLOSED
4,The Collective,"270 Rossiter Road, Koo Wee Rup",-38.199690,145.490323,Mon - Tue 10am-4pm,Fri - Sun 10am-4pm,NaN
5,All Nations Christian Church Op Shop,"170 Seacombe Rd, Seaview Downs",-35.029295,138.541582,Wed - Thu 10-4pm,NaN,NaN


#### Full-data

In [54]:
df_fulldata['address'].value_counts()

address
145 Nelson Street, Wallsend                                   2
25 Old Pacific  Highway, Yatala QLD 4207                      2
15 Main Rd, Moonah                                            2
1113 Riversdale Road, Surrey Hills                            2
271 Centre Road Bentleigh                                     2
290 Rossiter Road, Koo Wee Rup                                1
42 McBride Avenue, Wonthaggi, 3996                            1
2/36 Hartnett Dr 3198 Seaford, Victoria                       1
270 Rossiter Road, Koo Wee Rup                                1
170 Seacombe Rd, Seaview Downs                                1
6 jervois Street Albert park                                  1
6 Prossers Forest Road, Ravenswood, Launceston, TAS, 7250     1
165 City Walk, Canberra ACT 2600                              1
82 Limestone Avenue, Ainslie                                  1
41-43 Colbee court Phillip                                    1
48 Botany Street, Phillip       

**Findings**  
We can see the majority of the data are in the similar format of `Building Number`, `Street Name`, `City/Suburb`, `State Initial`, `Post Code`.  
Some of the address are not complete which can be indicate that the geolocation generated are not correct. -> [Geolocation Correction](#geolocation-correction)    
   
**Issues**   
Another issues appear where one address appear on more than one records, require further checks.

In [57]:
# List Non-unique Addresses
nonunique_addresses = df_fulldata['address'].value_counts()[df_fulldata['address'].value_counts() > 1].index.tolist()

# Display Dataframe with Non-unique Addresses
df_fulldata[df_fulldata['address'].isin(nonunique_addresses)]

,name,address,latitude,longitude,hours_1,hours_2,hours_3
235,Samaritans Wallsend,"145 Nelson Street, Wallsend",-32.897886,151.663566,Mon - Fri 9:30am - 4:30pm,Sat 9:30am - 12:30pm,NaN
237,Second Edition,"145 Nelson Street, Wallsend",-32.897886,151.663566,NaN,NaN,NaN
766,Yatala,"25 Old Pacific Highway, Yatala QLD 4207",-27.731458,153.222656,Mon - Fri 9-4pm,Sat 9-1pm,Sun Closed
767,Yatala,"25 Old Pacific Highway, Yatala QLD 4207",-27.731458,153.222656,Mon - Fri 9-4pm,Sat 9-1pm,Sun Closed
1401,Hobart City Mission - Moonah Mega-Store,"15 Main Rd, Moonah",-42.843104,147.292286,Mon - Fri 9.30am to 5.00pm,Sat 9:30am to 5:00pm,NaN
1403,City Mission Op Shops - Moonah Clothing,"15 Main Rd, Moonah",-42.843104,147.292286,Mon - Thu 10am to 5pm,Fri 10am to 9pm,Sat 10am to 5pm
1593,Rotary Recycle,"1113 Riversdale Road, Surrey Hills",-37.837421,145.110534,Mon - Fri 10am - 4pm,Sat 10am - 1pm,Sun Closed
1596,Rotary Recycle,"1113 Riversdale Road, Surrey Hills",-37.837421,145.110534,Mon - Fri 10am-4pm,Sat 10am - 1pm,Sun Closed
1746,Sacred Heart Mission,271 Centre Road Bentleigh,-37.917299,145.032710,Mon - Sun 10am - 5pm,NaN,NaN
1748,Sacred Heart Mission,271 Centre Road Bentleigh,-37.917299,145.032710,Mon - Sun 10am-5pm,NaN,NaN


**Findings**   
*  Some of the records are doubled (`Rotary Recycle`, `Yatala`, and `Sacred Heart Mission`) with subtle or no differences at all in the `hours_1` (e.g. for `Sacred Hearth Mission` the `hours_1` are only different in space before and after the opening hours).   
   
*  There are two different Op Shop with the same address ("Samaritans Wallsend" and "Second Edition"). It is indication of input error in the web display, so for now just just leave it as it is. -> [Community Apps Feature](#community-apps-feature) 

In [ ]:
# Deduplicate similar Op-Shops
df_fulldata = df_fulldata.drop_duplicates(
    subset=["name", "address", "latitude", "longitude"],
    keep="first"
).reset_index(drop=True)

# Display New DataFrame after Deduplication
nonunique_addresses = df_fulldata['address'].value_counts()[df_fulldata['address'].value_counts() > 1].index.tolist()

df_fulldata[df_fulldata['address'].isin(nonunique_addresses)]

,name,address,latitude,longitude,hours_1,hours_2,hours_3
176,Samaritans Wallsend,"145 Nelson Street, Wallsend",-32.897886,151.663566,Mon - Fri 9:30am - 4:30pm,Sat 9:30am - 12:30pm,NaN
178,Second Edition,"145 Nelson Street, Wallsend",-32.897886,151.663566,NaN,NaN,NaN
1046,Hobart City Mission - Moonah Mega-Store,"15 Main Rd, Moonah",-42.843104,147.292286,Mon - Fri 9.30am to 5.00pm,Sat 9:30am to 5:00pm,NaN
1048,City Mission Op Shops - Moonah Clothing,"15 Main Rd, Moonah",-42.843104,147.292286,Mon - Thu 10am to 5pm,Fri 10am to 9pm,Sat 10am to 5pm


The duplicate op-shops already dropped and the different name with same address is kept.

In [ ]:
df_fulldata[['no_building_number','street','city_suburb','state','postcode']] = df_fulldata['address'].str.split(',', expand=True)

#### Lat/Long = NaN

In [62]:
# Lat/Long = NaN
df_nan = df_opshop[
    (df_opshop["latitude"].isna()) | (df_opshop["longitude"].isna())
]

df_nan

,name,address,latitude,longitude,hours_1,hours_2,hours_3
2,The Green Shed Underground,"Basement level, 34 E Row Canberra ACT 2601 Australia",NaN,NaN,Tue - Fri 10:30am - 6pm,Sat 10:30am - 4pm,NaN
6,Anglican Op Shop,"Shop 1 100 - 106 Old Pacific Hwy, Oxenford 4210",NaN,NaN,Mon - Fri 8.30 - 4.30,Sat 8.30 -1,Sun Closed
8,Beach End Op Shop Mornington,70 McLaren Place Mornington Victoria 3931 Australia,NaN,NaN,Mon - Fri 10am - 4pm,Sat 10am - 2pm,Sun CLOSED
12,"1-9 Woolley Street, Dickson ACT 2602",Mon - Wed 9:00 am - 5:00pm,NaN,NaN,Thu - Sat 9:00 am - 8:30pm,Sun 9:00 am - 4:00pm,NaN
13,"Boolimba Crescent, Narrabundah ACT 2604",Mon - Tue 9.00 am - 5.00 pm,NaN,NaN,Wed - Fri 9:00 am - 5:00 pm,Sat 9.00 am - 3.00 pm,NaN
14,Calvary Chapel Opshop,"48-60 Cooper Street, Chifley ACT",NaN,NaN,Sat 8:00 am - 1:00pm,NaN,NaN
20,Weston Salvos,"14 - 20 Trennery St, Weston",NaN,NaN,Mon - Fri 9am to 5pm,Sat 9am - 3:30pm,NaN
22,"Cnr Ross Smith Cres and McIntosh Street, Scullin",Mon - Fri 10.00 am to 4.30 pm,NaN,NaN,Sat 10.00 am to 1.00 pm,NaN,NaN
24,kippax fair hardwick crescent holt,Mon - Fri 9.00 am - 5.00 pm,NaN,NaN,Sat 9:00 am - 4:00 pm,Sun 9:00 am - 4:00pm,NaN
26,"cnr Rae & Purdue Streets, Belconnen ACT 2617",Mon - Fri 9:00 am- 5:00 pm,NaN,NaN,Sat 9:00 am - 5:00 pm,Sun 9:00 am - 4:00 pm,NaN


### Opening Hours

# Data Exploration

## Data Overview

In [17]:
df_opshop.info()

<class 'pandas.DataFrame'>
RangeIndex: 2297 entries, 0 to 2296
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       2297 non-null   str    
 1   address    2297 non-null   str    
 2   latitude   1708 non-null   float64
 3   longitude  1708 non-null   float64
 4   hours_1    2123 non-null   str    
 5   hours_2    1671 non-null   str    
 6   hours_3    777 non-null    str    
dtypes: float64(2), str(5)
memory usage: 125.7 KB


Most of the data are in string as it was details of the Op Shops, only latitude and longitude are numeric that indicate the geolocation.   
   
There are **2,296** Op Shops across Australia recorded and scraped from opshops.org.

In [25]:
# Check Missing Value in Geolocation Data
print(f"Empty Latitude : {df_opshop['latitude'].isna().sum()}")
print(f"Empty Longitude : {df_opshop['longitude'].isna().sum()}")

Empty Latitude : 589
Empty Longitude : 589


There are quite a lot of op shops that not getting geolocation data correctly.   
Accounted ~25% of the whole data.   
   
The dataset still can be useful for exploration using other data that not use exact geolocation (e.g. Map Visualization).